If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [249]:
! pip install datasets transformers seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [250]:
from huggingface_hub import login

login("your_token")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/nikolaystepanov/.cache/huggingface/token
Login successful


Then you need to install Git-LFS. Uncomment the following instructions:

In [251]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [252]:
import transformers

print(transformers.__version__)

4.38.2


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/token-classification).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [253]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

# Fine-tuning a model on a token classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a token classification task, which is the task of predicting a label for each token.

![Widget inference representing the NER task](https://github.com/huggingface/notebooks/blob/main/examples/images/token_classification.png?raw=1)

The most common token classification tasks are:

- NER (Named-entity recognition) Classify the entities in the text (person, organization, location...).
- POS (Part-of-speech tagging) Grammatically classify the tokens (noun, verb, adjective...)
- Chunk (Chunking) Grammatically classify the tokens and group them into "chunks" that go together

We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [254]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "ai-forever/ruBert-base"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [255]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf). The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [256]:
datasets = load_dataset("iluvvatar/NEREL", trust_remote_code=True)

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [257]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'entities', 'relations', 'links'],
        num_rows: 746
    })
    test: Dataset({
        features: ['id', 'text', 'entities', 'relations', 'links'],
        num_rows: 93
    })
    dev: Dataset({
        features: ['id', 'text', 'entities', 'relations', 'links'],
        num_rows: 94
    })
})

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

In [258]:
datasets["train"][0]

{'id': 0,
 'text': 'Пулеметы, автоматы и снайперские винтовки изъяты в арендуемом американцами доме в Бишкеке\n\n05/08/2008 10:35\n\nБИШКЕК, 5 августа /Новости-Грузия/. Правоохранительные органы Киргизии обнаружили в доме, арендуемом гражданами США в Бишкеке, пулеметы, автоматы и снайперские винтовки, сообщает во вторник пресс-служба МВД Киргизии.\n\n"В ходе проведения оперативно-профилактического мероприятия под кодовым названием "Арсенал" в новостройке Ынтымак, в доме, принадлежащем 66-летнему гражданину Киргизии и арендуемом гражданами США, обнаружены и изъяты: шесть крупнокалиберных пулеметов с оптическим прицелом и с приборами ночного видения, 26 автоматов калибра 5,56 миллиметра, два винчестера марки МОСВЕГА 12-го калибра, четыре ствола от крупнокалиберного пулемета, два подствольных гранатомета, четыре снайперские винтовки с оптическим прицелом защитного цвета, шесть пистолетов калибра 9 миллиметров марки Беретта, одна винтовка", - говорится в сообщении МВД.\n\nПресс-служба отме

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [259]:
import re

def transform_entities(entities):
    markup = []
    for i in entities:
        splited = re.split('\t', i)
        data = re.split(' ', splited[1])
        if ";" in data[2]:
            data[2] = data[2].split(";")[0]
        markup.append({'id': splited[0],
               'type': data[0],
               'start': int(data[1]),
               'stop': int(data[2]),
               'text': splited[2]})
    markup.sort(key=lambda x: x["start"])
    return markup

In [260]:
transform_entities(datasets["train"][0]['entities'])

[{'id': 'T1',
  'type': 'NATIONALITY',
  'start': 62,
  'stop': 74,
  'text': 'американцами'},
 {'id': 'T2', 'type': 'CITY', 'start': 82, 'stop': 89, 'text': 'Бишкеке'},
 {'id': 'T11', 'type': 'DATE', 'start': 91, 'stop': 101, 'text': '05/08/2008'},
 {'id': 'T69',
  'type': 'TIME',
  'start': 91,
  'stop': 107,
  'text': '05/08/2008 10:35'},
 {'id': 'T71', 'type': 'CITY', 'start': 109, 'stop': 115, 'text': 'БИШКЕК'},
 {'id': 'T3', 'type': 'DATE', 'start': 117, 'stop': 126, 'text': '5 августа'},
 {'id': 'T72',
  'type': 'ORGANIZATION',
  'start': 128,
  'stop': 142,
  'text': 'Новости-Грузия'},
 {'id': 'T4', 'type': 'COUNTRY', 'start': 136, 'stop': 142, 'text': 'Грузия'},
 {'id': 'T5',
  'type': 'ORGANIZATION',
  'start': 145,
  'stop': 179,
  'text': 'Правоохранительные органы Киргизии'},
 {'id': 'T6',
  'type': 'COUNTRY',
  'start': 171,
  'stop': 179,
  'text': 'Киргизии'},
 {'id': 'T104',
  'type': 'NATIONALITY',
  'start': 210,
  'stop': 224,
  'text': 'гражданами США'},
 {'id': 'T

In [261]:
def transform(text, markup): 
    tokens = [text[0:markup[0]['start']]]
    tags = ['O']
    
    for i in range(len(markup[:-1])):
        tokens.append(text[markup[i]['start']:markup[i]['stop']])
        tags.append(markup[i]['type'])
        tokens.append(text[markup[i]['stop']:markup[i + 1]['start']])
        tags.append('O')

    tokens.append(text[markup[-1]['start']:markup[-1]['stop']])
    tags.append(markup[-1]['type'])
    tokens.append(text[markup[-1]['stop']:])
    tags.append('O')
    
    final_tokens = []
    final_tags = []
    
    for i in range(len(tokens)):
        size = len(tokens[i].split())
        final_tokens += tokens[i].split()
        if tags[i] != "O":
            final_tags.append("B-" + tags[i])
            final_tags += ["I-" + tags[i]] * (size - 1)
        else:
            final_tags += [tags[i]] * size
        
    return final_tokens, final_tags

In [262]:
tokens, entities = transform(datasets["train"][0]['text'], transform_entities(datasets["train"][0]['entities']))

In [263]:
len(tokens), len(entities)

(445, 445)

In [264]:
entities

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NATIONALITY',
 'O',
 'O',
 'B-CITY',
 'B-DATE',
 'B-TIME',
 'I-TIME',
 'B-CITY',
 'O',
 'B-DATE',
 'I-DATE',
 'O',
 'B-ORGANIZATION',
 'B-COUNTRY',
 'O',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'I-ORGANIZATION',
 'B-COUNTRY',
 'O',
 'O',
 'O',
 'O',
 'B-NATIONALITY',
 'I-NATIONALITY',
 'B-COUNTRY',
 'O',
 'B-CITY',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DATE',
 'I-DATE',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'I-ORGANIZATION',
 'B-ORGANIZATION',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'B-COUNTRY',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-EVENT',
 'O',
 'O',
 'O',
 'B-CITY',
 'O',
 'O',
 'O',
 'O',
 'B-AGE',
 'B-NATIONALITY',
 'I-NATIONALITY',
 'B-COUNTRY',
 'O',
 'O',
 'B-NATIONALITY',
 'I-NATIONALITY',
 'B-COUNTRY',
 'O',
 'B-EVENT',
 'I-EVENT',
 'I-EVENT',
 'O',
 'B-NUMBER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NUMBER',
 'O',
 'O',
 'B-NUMBER',
 'I-NUMBER',
 'O',
 'B-NUMBER',
 '

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:
- 'PER' for person
- 'ORG' for organization
- 'LOC' for location
- 'MISC' for miscellaneous

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [265]:
import torch


class OwnDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        item = self.dataset[idx]
        text = item['text']
        tokens, entities = transform(item['text'], transform_entities(item['entities']))
        return {"text": text,
                "tokens": tokens,
                "ner_tags": entities}

In [266]:
train_dataset = OwnDataset(datasets['train'])

In [267]:
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(columns=["text", "tokens", "ner_tags"])
    for idx in range(num_examples):
        df.at[idx, "text"] = dataset[picks[idx]]["text"]
        df.at[idx, "tokens"] = dataset[picks[idx]]["tokens"]
        df.at[idx, "ner_tags"] = dataset[picks[idx]]["ner_tags"]
    display(HTML(df.to_html()))

In [268]:
show_random_elements(train_dataset)

,text,tokens,ner_tags
0,"Две лесбиянки со взрослым сыном из России поженились и попросили политического убежища в Аргентине\nВ день четырёхлетия принятия закона об однополом браке в Аргентине — 15 июля 2014 года, в Буэнос-Айресе две российские лесбиянки — Марина Миронова и Оксана Тимофеева, воспитывающие 16-летноего сына, поженились и пропросили политического убежища в этой стране.\nОб этом ообщает агентство EFE со ссылкой на Федерацию лесбиянок, геев, бисексуалов и транссексуалов Аргентины (FALGBT).\n\nЧерез четыре года после принятия закона об однополом браке в Аргентине, две русские женщины женились в Буэнос-Айресе, 15 июля 2014 года. Источник видео.\n\nЖенщины заявили, что дата совпала случайно, а их выбор пал на Аргентину, так как они хотят жить «свободно и в безопасности», и эта страна может предоставить им такую возможность.\n\nВице-президент FALGBT Клаудия Кастросин Верду, ставшая свидетелем на свадьбе, пояснила, что пара искала помощи у аргентинской федерации, страдая от преследований гомосексуалистов в России, начавшихся в конце 2012 года, после принятия закона о запрете пропаганды гомосексуализма среди несовершеннолетних:\nУ них есть 16-летний сын, которого они не хотят потерять.\n\nКастросин Верду также выразила обеспокоенность ущемлением в России прав сексуальных меньшинств.\n\nЖенщины 38 и 36 лет приехали в Аргентину месяц назад после того, как узнали о якобы готовящемся законе, по которому у однополых пар смогут отбирать детей.\nСМИ отмечают, что они фактически бежали из России в Аргентину, не зная в должной мере ни английского ни испанского языка.\n\nМарина Миронова (мать 16-летнего парня) также сообщила изданию Télam, что в России она работала учителем и была уволена, после того как в школе узнали о том, что она живёт с женщиной.\n\nС конца 2012 года, когда в России был принят «антигейский» закон, давление на лиц «нетрадиционной» ориентации только усиливается.\n\nИм запрещают проводить акции и шествия в защиту своих прав.\n\nДикторы российского телевидения допускают оскорбительные и провокационные высказывания в адрес ЛГБТ-сообщества, предлагая сжигать в печах «сердца геев» и их самих.\n\nВ сентябре 2013 года депутат Госдумы РФ от Единой России, председатель политической партии «Родина» Алексей Журавлёв выступил инициатором законопроекта о лишении родительских прав лиц нетрадиционной сексуальной ориентации.\nДокладчик ПАСЕ по правам ЛГБТ Х. Хаугли расценил эту инициативу как проявление гомофобной пропаганды, противоречащее международным обязательствам России.\n","[Две, лесбиянки, со, взрослым, сыном, из, России, поженились, и, попросили, политического, убежища, в, Аргентине, В, день, четырёхлетия, принятия, закона, закона, об, однополом, браке, в, Аргентине, —, 15, июля, 2014, года, ,, в, Буэнос-Айресе, две, российские, лесбиянки, —, Марина, Миронова, и, Оксана, Тимофеева, ,, воспитывающие, 16-летноего, сына,, поженились, и, пропросили, политического, убежища, в, этой, стране., Об, этом, ообщает, агентство, EFE, со, ссылкой, на, Федерацию, лесбиянок,, геев,, бисексуалов, и, транссексуалов, Аргентины, Аргентины, (, FALGBT, )., Через, четыре, года, после, принятия, закона, об, однополом, браке, закона, об, однополом, браке, в, Аргентине, ,, две, русские, женщины, женились, в, Буэнос-Айресе, ,, 15, июля, 2014, года, ...]","[B-NUMBER, B-EVENT, O, O, O, O, B-COUNTRY, O, O, O, O, O, O, B-COUNTRY, B-DATE, I-DATE, B-AGE, B-EVENT, I-EVENT, B-LAW, I-LAW, I-LAW, I-LAW, O, B-COUNTRY, O, B-DATE, I-DATE, I-DATE, I-DATE, O, O, B-CITY, B-NUMBER, B-NATIONALITY, O, O, B-PERSON, I-PERSON, O, B-PERSON, I-PERSON, O, O, B-AGE, O, B-EVENT, O, B-EVENT, I-EVENT, I-EVENT, O, O, O, O, O, O, O, B-ORGANIZATION, O, O, O, B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, B-COUNTRY, O, B-ORGANIZATION, O, B-DATE, I-DATE, I-DATE, O, B-EVENT, I-EVENT, I-EVENT, I-EVENT, I-EVENT, B-LAW, I-LAW, I-LAW, I-LAW, O, B-COUNTRY, O, B-NUMBER, B-NATIONALITY, O, B-EVENT, O, B-CITY, O, B-DATE, I-D

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [269]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [270]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [271]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 67124, 70471, 121, 26802, 13218, 30046, 118080, 12894, 177, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [272]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 67124, 70471, 121, 26802, 13218, 30046, 118080, 12894, 16994, 69821, 443, 42038, 119660, 454, 126, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [273]:
example = train_dataset[4]
print(example["tokens"])

['Facebook', 'нашел', 'нового', 'финансового', 'директора', 'Финансовым', 'директором', 'социальной', 'сети', 'Facebook', 'назначен', '39-летний', 'Дэвид', 'Эберсман', '(', 'David', 'Ebersman', '),', 'сообщает', 'The', 'Wall', 'Street', 'Journal', '.', 'На', 'работу', 'в', 'Facebook', 'он', 'выйдет', 'в', 'сентябре', '.', 'Ранее', 'Эберсман', 'был', 'финансовым', 'директором', 'биотехнологической', 'компании', 'Genentech', '.', 'Эберсман', 'подчеркнул,', 'что', 'видит', 'много', 'общего', 'между', 'Facebook', 'и', 'Genentech', '.', 'В', 'частности,', 'это', 'две', 'быстрорастущие', 'компании', 'с', 'сильной', 'корпоративной', 'культурой.', 'Также', 'он', 'заявил,', 'что', 'Facebook', 'ожидает', '70-процентное', 'увеличение', 'выручки', 'в', '2009', 'году', '.', 'В', 'компании', 'Genentech', 'Дэвид', 'Эберсман', 'проработал', '15', 'лет', '.', 'Ее', 'финансовым', 'директором', 'он', 'стал', 'в', '2006', 'году', '.', 'На', 'этой', 'должности', 'Эберсман', 'проработал', 'до', 'апреля', '2

In [274]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'facebook', 'нашел', 'нового', 'финансового', 'директора', 'финансовым', 'директором', 'социально', '##и', 'сети', 'facebook', 'назначен', '39', '-', 'лет', '##нии', 'дэ', '##вид', 'э', '##бер', '##сман', '(', 'dav', '##id', 'e', '##ber', '##sm', '##an', ')', ',', 'сообщает', 'the', 'w', '##all', 'str', '##ee', '##t', 'j', '##ournal', '.', 'на', 'работу', 'в', 'facebook', 'он', 'выи', '##дет', 'в', 'сентябре', '.', 'ранее', 'э', '##бер', '##сман', 'был', 'финансовым', 'директором', 'биотехнологи', '##ческо', '##и', 'компании', 'gen', '##ente', '##ch', '.', 'э', '##бер', '##сман', 'подчеркнул', ',', 'что', 'видит', 'много', 'общего', 'между', 'facebook', 'и', 'gen', '##ente', '##ch', '.', 'в', 'частности', ',', 'это', 'две', 'быстрорасту', '##щие', 'компании', 'с', 'сильно', '##и', 'корпора', '##тивно', '##и', 'культуро', '##и', '.', 'также', 'он', 'заявил', ',', 'что', 'facebook', 'ожидает', '70', '-', 'процент', '##ное', 'увеличение', 'выручки', 'в', '2009', 'году', '.', 'в'

Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [275]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(199, 283)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [276]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 11, 12, 12, 13, 13, 13, 14, 15, 15, 16, 16, 16, 16, 17, 17, 18, 19, 20, 20, 21, 21, 21, 22, 22, 23, 24, 25, 26, 27, 28, 29, 29, 30, 31, 32, 33, 34, 34, 34, 35, 36, 37, 38, 38, 38, 39, 40, 40, 40, 41, 42, 42, 42, 43, 43, 44, 45, 46, 47, 48, 49, 50, 51, 51, 51, 52, 53, 54, 54, 55, 56, 57, 57, 58, 59, 60, 60, 61, 61, 61, 62, 62, 62, 63, 64, 65, 65, 66, 67, 68, 69, 69, 69, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 78, 78, 79, 79, 80, 80, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 95, 96, 97, 97, 97, 98, 99, 100, 101, 102, 103, 104, 105, 105, 106, 106, 106, 107, 108, 108, 108, 109, 110, 111, 112, 113, 113, 114, 114, 114, 115, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 127, 128, 128, 129, 130, 131, 132, 133, 134, 135, 136, 136, 137, 137, 137, 138, 139, 140, 140, 140, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 155, 155, 156, 157, 158, 158, 158, 159, 159, 160, 1

As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [277]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))
for i in range(len(aligned_labels)):
    print(*tokenizer.convert_ids_to_tokens([tokenized_input["input_ids"][i]]), aligned_labels[i])

283 283
[CLS] -100
facebook B-ORGANIZATION
нашел O
нового O
финансового B-PROFESSION
директора I-PROFESSION
финансовым B-PROFESSION
директором I-PROFESSION
социально O
##и O
сети O
facebook B-ORGANIZATION
назначен B-EVENT
39 B-AGE
- B-AGE
лет B-AGE
##нии B-AGE
дэ B-PERSON
##вид B-PERSON
э I-PERSON
##бер I-PERSON
##сман I-PERSON
( O
dav B-PERSON
##id B-PERSON
e I-PERSON
##ber I-PERSON
##sm I-PERSON
##an I-PERSON
) O
, O
сообщает O
the B-ORGANIZATION
w I-ORGANIZATION
##all I-ORGANIZATION
str I-ORGANIZATION
##ee I-ORGANIZATION
##t I-ORGANIZATION
j I-ORGANIZATION
##ournal I-ORGANIZATION
. O
на O
работу O
в O
facebook B-ORGANIZATION
он O
выи O
##дет O
в B-DATE
сентябре I-DATE
. O
ранее O
э B-PERSON
##бер B-PERSON
##сман B-PERSON
был O
финансовым B-PROFESSION
директором I-PROFESSION
биотехнологи O
##ческо O
##и O
компании O
gen B-ORGANIZATION
##ente B-ORGANIZATION
##ch B-ORGANIZATION
. O
э B-PERSON
##бер B-PERSON
##сман B-PERSON
подчеркнул O
, O
что O
видит O
много O
общего O
между O
faceboo

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [278]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [279]:
label2id = {
    'O': 0,
    'B-AGE': 1,
    'I-AGE': 2,
    'B-AWARD': 3,
    'I-AWARD': 4,
    'B-CITY': 5,
    'I-CITY': 6,
    'B-COUNTRY': 7,
    'I-COUNTRY': 8,
    'B-CRIME': 9,
    'I-CRIME': 10,
    'B-DATE': 11,
    'I-DATE': 12,
    'B-DISEASE': 13,
    'I-DISEASE': 14,
    'B-DISTRICT': 15,
    'I-DISTRICT': 16,
    'B-EVENT': 17,
    'I-EVENT': 18,
    'B-FACILITY': 19,
    'I-FACILITY': 20,
    'B-FAMILY': 21,
    'I-FAMILY': 22,
    'B-IDEOLOGY': 23,
    'I-IDEOLOGY': 24,
    'B-LANGUAGE': 25,
    'I-LANGUAGE': 26,
    'B-LAW': 27,
    'I-LAW': 28,
    'B-LOCATION': 29,
    'I-LOCATION': 30,
    'B-MONEY': 31,
    'I-MONEY': 32,
    'B-NATIONALITY': 33,
    'I-NATIONALITY': 34,
    'B-NUMBER': 35,
    'I-NUMBER': 36,
    'B-ORDINAL': 37,
    'I-ORDINAL': 38,
    'B-ORGANIZATION': 39,
    'I-ORGANIZATION': 40,
    'B-PENALTY': 41,
    'I-PENALTY': 42,
    'B-PERCENT': 43,
    'I-PERCENT': 44,
    'B-PERSON': 45,
    'I-PERSON': 46,
    'B-PRODUCT': 47,
    'I-PRODUCT': 48,
    'B-PROFESSION': 49,
    'I-PROFESSION': 50,
    'B-RELIGION': 51,
    'I-RELIGION': 52,
    'B-STATE_OR_PROVINCE': 53,
    'I-STATE_OR_PROVINCE': 54,
    'B-TIME': 55,
    'I-TIME': 56,
    'B-WORK_OF_ART': 57,
    'I-WORK_OF_ART': 58,
}

In [280]:
def tokenize_and_align_labels(examples):
    
    transformed = {"id": [], "text": [], "tokens": [], "ner_tags": []}
    
    for idx, text, entities in zip(examples["id"], examples["text"], examples["entities"]):
        tokens, ner_tags = transform(text, transform_entities(entities))
        transformed["id"].append(idx)
        transformed["text"].append(text)
        transformed["tokens"].append(tokens)
        transformed["ner_tags"].append(ner_tags)
    
    tokenized_inputs = tokenizer(transformed["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(transformed[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [281]:
tokenize_and_align_labels(datasets["train"][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 56081, 121, 37009, 107, 3594, 378, 1004, 1022, 29653, 36666, 113, 15568, 10060, 35797, 4308, 113, 1461, 4356, 809, 9381, 197, 8535, 197, 4216, 9381, 197, 8535, 197, 4216, 1355, 162, 3902, 1461, 4356, 385, 121, 167, 2684, 197, 5880, 133, 19361, 390, 19361, 390, 197, 126, 19893, 7528, 73689, 54896, 73689, 54896, 8201, 113, 4308, 121, 15568, 10060, 21578, 67899, 67899, 113, 1461, 4356, 809, 121, 56081, 121, 37009, 107, 3594, 378, 1004, 1022, 29653, 121, 1762, 703, 6697, 2262, 133, 4167, 115, 384, 387, 73689, 54896, 115, 384, 387, 115, 384, 387, 73689, 54896, 73689, 54896, 126, 152, 113, 2972, 5303, 14455, 133, 22294, 2008, 6824, 771, 89786, 6288, 152, 44732, 152, 113, 2257, 6569, 809, 120, 24042, 23009, 121, 113, 4308, 121, 94152, 9485, 133, 28225, 49857, 73689, 54896, 73689, 54896, 107, 15568, 10060, 21578, 67899, 67899, 121, 14204, 107, 36666, 162, 4439, 89209, 41096, 110, 7564, 3985, 52226, 107, 110, 62046, 26369, 26998, 121, 2900, 33131, 19842, 167, 121, 7889, 608

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [282]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [283]:
from transformers import BertForTokenClassification, AutoConfig, TrainingArguments, Trainer

config = AutoConfig.from_pretrained(model_checkpoint)

id2label = {y: x for x, y in label2id.items()}
config.label2id = label2id
config.id2label = id2label
config._num_labels = len(label2id)

model = BertForTokenClassification.from_pretrained(model_checkpoint, config=config)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [284]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [285]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [286]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [287]:
metric = load_metric("seqeval")

/Users/nikolaystepanov/PycharmProjects/NLP/venv/lib/python3.9/site-packages/datasets/load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


This metric takes list of labels for the predictions and references:

In [288]:
labels = [i for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'AGE': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'DATE': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 7},
 'EVENT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'LOCATION': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'NUMBER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'ORGANIZATION': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 16},
 'PERCENT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PERSON': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 10},
 'PROFESSION': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 10},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [289]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [290]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/Users/nikolaystepanov/PycharmProjects/NLP/venv/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


We can now finetune our model by just calling the `train` method:

In [291]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [292]:
trainer.train()

RuntimeError: The size of tensor a (1480) must match the size of tensor b (512) at non-singleton dimension 1

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

{'eval_loss': 0.05934586375951767,
 'eval_precision': 0.9292672127518264,
 'eval_recall': 0.9391430808815304,
 'eval_f1': 0.9341790463472988,
 'eval_accuracy': 0.9842565968195466,
 'epoch': 3.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': 0.949718574108818,
  'recall': 0.966768525592055,
  'f1': 0.9581677077418134,
  'number': 2618},
 'MISC': {'precision': 0.8132387706855791,
  'recall': 0.8383428107229894,
  'f1': 0.8255999999999999,
  'number': 1231},
 'ORG': {'precision': 0.9055232558139535,
  'recall': 0.9090466926070039,
  'f1': 0.9072815533980583,
  'number': 2056},
 'PER': {'precision': 0.9759552042160737,
  'recall': 0.9765985497692815,
  'f1': 0.9762767710049424,
  'number': 3034},
 'overall_precision': 0.9292672127518264,
 'overall_recall': 0.9391430808815304,
 'overall_f1': 0.9341790463472988,
 'overall_accuracy': 0.9842565968195466}

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("sgugger/my-awesome-model")
```